In [82]:
import pandas as pd
import numpy as np
import geopy.distance
from math import radians, cos, sin, asin, sqrt

In [4]:
import os
import sys

src_dir = os.path.join(os.getcwd(), "..", "..")

sys.path.append(os.path.abspath(src_dir))

from src.helpers.city_to_train_station import VisualizerHelper

In [8]:
train_station_df = pd.read_csv('../../../data/liste-des-gares.csv', sep=';')
cities_df = pd.read_csv('../../../data/cities.csv')

city_names_df = cities_df["label"]

In [ ]:
coord_1 = (52.2296756, 21.0122287)
coord_2 = (52.406374, 16.9251681)

distance = geopy.distance.distance(coord_1, coord_2).km

print(distance)

In [118]:
def haversine_vectorized(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))

    r = 6371

    distance = c * r
    return distance

def get_nearest_station(city_name):
    city = cities_df.loc[cities_df["label"] == city_name]
    coord_1 = (city["latitude"].values[0], city["longitude"].values[0])

    train_station_df[['lat', 'lon']] = train_station_df['C_GEO'].str.split(',', expand=True).astype(float)

    train_station_df['distance'] = haversine_vectorized(
        coord_1[0], coord_1[1],
        train_station_df['lat'].values, train_station_df['lon'].values
    )

    distance_df = train_station_df[['distance', 'LIBELLE']].copy()
    return distance_df.loc[distance_df["distance"].min() == distance_df["distance"]]


In [121]:
cities_df.sample(5)

,insee_code,city_code,zip_code,label,latitude,longitude,department_name,department_number,region_name,region_geojson_name
30953,68190,lucelle,68480,lucelle,47.438361,7.276558,haut-rhin,68,grand est,Grand Est
21608,32390,st martin d armagnac,32110,st martin d armagnac,43.711146,-0.063891,gers,32,occitanie,Occitanie
4840,38374,st chef,38890,st chef,45.641242,5.361508,isère,38,auvergne-rhône-alpes,Auvergne-Rhône-Alpes
21963,03127,herisson,3190,herisson,46.519794,2.710394,allier,03,auvergne-rhône-alpes,Auvergne-Rhône-Alpes
39073,98723,hiva oa,98741,nahoe,NaN,NaN,polynésie-française,987,polynésie-française,Polynésie française


In [123]:
get_nearest_station("lucelle")

,distance,LIBELLE
2947,21.175663,Altkirch


In [74]:
city = cities_df.loc[cities_df["label"] == "ville du pont"]
coord_1 = (city["latitude"].values[0], city["longitude"].values[0])

distance_df = []
for row in train_station_df.iterrows():
    coord_2 = row[1].C_GEO
    lat = float(coord_2.split(",")[0])
    lon = float(coord_2.split(",")[1])
    coord_2 = (lat, lon)
    distance = geopy.distance.distance(coord_1, coord_2).km
    distance_df.append({
        "distance": distance,
        "row": row[1].LIBELLE
    })

distance_df = pd.DataFrame(distance_df)
distance_df.min()

In [23]:
train_station_df.loc[train_station_df["COMMUNE"].str.contains(" ")]

,CODE_UIC,LIBELLE,FRET,VOYAGEURS,CODE_LIGNE,RG_TRONCON,PK,COMMUNE,DEPARTEMEN,IDRESEAU,IDGAIA,X_L93,Y_L93,X_WGS84,Y_WGS84,C_GEO,Geo Point,Geo Shape
11,87473181,Plouaret-Trégor,N,O,420000,1,530+923,LE VIEUX-MARCHE,COTES-D'ARMOR,6025,296e5ab0-dfbc-11e3-a2ff-01a464e0362d,224408.0606,6.853469e+06,-3.457984,48.606291,"48.606290948971115, -3.4579839535560173","48.60629094894776, -3.457983953556023","{""coordinates"": [-3.457983953556023, 48.606290..."
37,87746727,Vaudagne,N,O,896000,1,008+930,LES HOUCHES,HAUTE-SAVOIE,7112,2982b01c-dfbc-11e3-a2ff-01a464e0362d,992354.2991,6.541335e+06,6.772719,45.908833,"45.9088333354493, 6.7727189535977415","45.90883333543229, 6.772718953597744","{""coordinates"": [6.772718953597744, 45.9088333..."
39,87297085,La Bouteille,N,O,229000,1,186+430,LA BOUTEILLE,AISNE,4602,c101ed7c-f312-11e3-90ff-015864e0362d,770718.8958,6.973925e+06,3.982780,49.860132,"49.860132006253636, 3.982780234602766","49.86013200623647, 3.982780234602767","{""coordinates"": [3.982780234602767, 49.8601320..."
54,87747485,Le Cheylas-La Buissière,O,N,909000,1,036+673,LE CHEYLAS,ISERE,9397,d9dbfa4c-6667-11e3-89ff-01f464e0362d,934466.6179,6.481531e+06,5.997235,45.393135,"45.39313502704879, 5.997234672401361","45.39313502703032, 5.997234672401363","{""coordinates"": [5.997234672401363, 45.3931350..."
75,87783308,Le Monastier,N,O,722000,1,615+161,BOURGS SUR COLAGNE,LOZERE,4969,d9e429e6-6667-11e3-89ff-01f464e0362d,720049.1477,6.378903e+06,3.252222,44.508907,"44.50890669463762, 3.2522221240054483","44.50890669461671, 3.2522221240054483","{""coordinates"": [3.252222124005448, 44.5089066..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3820,87396036,La Guierche,N,O,430000,1,015+105,LA GUIERCHE,SARTHE,4685,292b1c54-dfbc-11e3-a2ff-01a464e0362d,491165.8399,6.783010e+06,0.193218,48.114054,"48.11405392276398, 0.19321777716732255","48.11405392275342, 0.19321777716732055","{""coordinates"": [0.193217777167321, 48.1140539..."
3825,87783266,Aumont-Aubrac,N,O,722000,1,641+898,PEYRE EN AUBRAC,LOZERE,2996,d96d99bc-6667-11e3-89ff-01f464e0362d,722440.5879,6.402159e+06,3.283357,44.718195,"44.71819524766465, 3.2833565153888573","44.71819524764431, 3.2833565153888573","{""coordinates"": [3.283356515388857, 44.7181952..."
3826,87691394,La Marche,N,O,750000,1,231+731,LA MARCHE,NIEVRE,4701,c1067856-f312-11e3-90ff-015864e0362d,702995.4585,6.671056e+06,3.039523,47.139767,"47.13976742721954, 3.0395231937195466","47.139767427206124, 3.0395231937195466","{""coordinates"": [3.039523193719547, 47.1397674..."
3845,87645507,Neussargues,O,O,720000,1,359+737,NEUSSARGUES EN PINATELLE,CANTAL,5767,c0e67c10-f312-11e3-90ff-015864e0362d,698358.6884,6.447333e+06,2.979124,45.125279,"45.12527901420377, 2.9791243255770485","45.12527901418452, 2.9791243255770485","{""coordinates"": [2.979124325577049, 45.1252790..."


In [2]:
VisualizerHelper.convert_city_to_train_station("Paris")